In [1]:
import pandas as pd
import os
import numpy as np
from scipy import stats as st

Прочитаем файлы для анализа

In [2]:
ab_test = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\AB Test Hit.csv', parse_dates=['hit_at'])
add_to_card = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Add To Cart Clicked.csv', parse_dates=['timestamp'])
address_change = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Address Change Initiated.csv', parse_dates=['timestamp'])
message_not_zone= pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Address Not In Delivery Zone.csv', parse_dates=['timestamp'])
landing_viewed = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Landing Viewed.csv', parse_dates=['timestamp'])
retailer = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Main Page Viewed.csv', parse_dates=['timestamp'])
order = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Order Completed.csv', parse_dates=['timestamp'])
product_add = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Product Added.csv', parse_dates=['timestamp'])
shop_started = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Shop Celection Started.csv', parse_dates=['timestamp'])
shop_selected = pd.read_csv('~\Documents\СберМаркет\Career Factory-20210428T114851Z-001\Career Factory\Shop Selected.csv', parse_dates=['timestamp'])


Ключом в каждой таблице будет id пользователя и дата посещени. Добавим информауию о дате в каждую таблицу

In [3]:
ab_test['date']=ab_test.hit_at.dt.to_period('D')

C:\Users\user\Anaconda3\1\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [4]:
landing_viewed['date'] = landing_viewed.timestamp.dt.to_period('D') # добавлем столбец с датой без времени

In [5]:
retailer['date'] = retailer.timestamp.dt.to_period('D')

In [6]:
address_change['date'] = address_change.timestamp.dt.to_period('D')

In [7]:
message_not_zone['date'] = message_not_zone.timestamp.dt.to_period('D')

In [8]:
product_add['date'] = product_add.timestamp.dt.to_period('D')

In [9]:
shop_started['date'] = shop_started.timestamp.dt.to_period('D')

In [10]:
shop_selected['date'] = shop_selected.timestamp.dt.to_period('D')

In [11]:
add_to_card['date'] = add_to_card.timestamp.dt.to_period('D')

In [12]:
order['date'] = order.timestamp.dt.to_period('D')

Данные по тому в какую группу попал клиент содержаться в таблице "ab_test". Проверим сколько клиентов попало в каждую группу и есть ли дубликаты.

In [13]:
client_group_count = ab_test.groupby(['anonymous_id', 'date', 'group'], as_index = False).agg({'hit_at': 'count'})\
                             .groupby(['anonymous_id', 'date'], as_index = False).agg({'group': 'count'})\
                             .rename(columns = {'group': 'group_count'})
print('Всего пользователей в обеих группах: {}'.format(client_group_count.shape[0]))
print('Из отраженных в обеих группах: {}'.format(client_group_count.query('group_count>1').shape[0]))

Всего пользователей в обеих группах: 501423
Из отраженных в обеих группах: 513


Удалим дубликаты в таблице "ab_test"

In [14]:
ab_test_not_dup = client_group_count.query('group_count < 2')\
                              .merge(ab_test, on = ['date', 'anonymous_id'], how = 'left')\
                              .sort_values('hit_at')\
                              .drop_duplicates(subset=['date', 'anonymous_id'], keep='last')    

In [15]:
ab_test_not_dup.duplicated(subset=['date', 'anonymous_id']).sum()  

0

Мы оставили только один вариант столбца device_type, browser и os. Проверим на сколько это принципиально. (расчеты для проверки отличий в ОС пользователей, зафиксированных в один день несколько раз.)

In [19]:
ab_test_not_dup_first = client_group_count.query('group_count < 2')\
                              .merge(ab_test, on = ['date', 'anonymous_id'], how = 'left')\
                              .sort_values('hit_at')\
                              .drop_duplicates(subset=['date', 'anonymous_id'], keep='first')
ab_test_not_dup['device_type'].value_counts()-ab_test_not_dup_first['device_type'].value_counts()
# Разница несущественная, для анализа будем учитывать последнюю зафиксированную ос, device_type и browser/

mobile    -17
desktop    17
tablet      0
console     0
smarttv     0
Name: device_type, dtype: int64

Проверим период эксперимента и количество участников в группах после удаления дубликатов

In [20]:
pd.pivot_table(ab_test_not_dup, values='group_count', index='date',
                    columns='group', aggfunc= 'count', margins=True).reset_index().rename_axis(None, axis=1)
#, margins=True

,date,address_first,default,All
0,2020-11-26,3722,29866,33588
1,2020-11-27,3630,29383,33013
2,2020-11-28,3659,29138,32797
3,2020-11-29,3388,26475,29863
4,2020-11-30,3155,25318,28473
5,2020-12-01,3118,24246,27364
6,2020-12-02,3650,28357,32007
7,2020-12-03,3698,29821,33519
8,2020-12-04,3796,30911,34707
9,2020-12-05,4122,32970,37092


Соберем в один файл информацию по количеству посещений страниц каждым пользователем

In [21]:
landing_count = landing_viewed.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_landing'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [22]:
retailer_count = retailer.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_mail'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [23]:
address_change_count = address_change.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_address_change'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [24]:
not_zone_count = message_not_zone.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_not_zone'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [25]:
product_add_count = product_add.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_product_add'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [26]:
shop_started_count = shop_started.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_shop_started'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')

In [27]:
shop_selected_count = shop_selected.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_shop_selected'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [28]:
add_to_card_count = add_to_card.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_add_to_card'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [29]:
order_count = order.groupby(['date','anonymous_id'], as_index = False).agg({'timestamp': 'count'})\
                        .rename(columns = {'timestamp': 'count_order'})\
                        .query('date < "2020-12-11" and date >= "2020-11-26"')


In [30]:
total_data = (
    ab_test_not_dup_first.merge(landing_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(retailer_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(address_change_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(not_zone_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(product_add_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(shop_started_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(shop_selected_count, on = ['date', 'anonymous_id'], how = 'left')    
                         .merge(add_to_card_count, on = ['date', 'anonymous_id'], how = 'left')
                         .merge(order_count, on = ['date', 'anonymous_id'], how = 'left')    
)

In [31]:
total_data.fillna(0, inplace=True)

In [32]:
total_data.tail()

,anonymous_id,date,group_count,hit_at,group,device_type,browser,os,count_landing,count_mail,count_address_change,count_not_zone,count_product_add,count_shop_started,count_shop_selected,count_add_to_card,count_order
500905,5e518207-82fb-48b9-9764-c167e84dc57c,2020-12-10,1,2020-12-10 23:59:18.618000+00:00,default,desktop,Chrome,Linux,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500906,90b60ec1-4d0e-4fab-8cf0-6c5c128ea5ec,2020-12-10,1,2020-12-10 23:59:35.345000+00:00,default,mobile,Mobile Safari,iOS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500907,454c7515-0a58-4822-a431-d271e91da190,2020-12-10,1,2020-12-10 23:59:36.293000+00:00,default,desktop,Chrome,Linux,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500908,ad4b596b-88d3-41e9-969a-7d988039405b,2020-12-10,1,2020-12-10 23:59:48.461000+00:00,default,desktop,Chrome,Windows,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500909,49ec0fa2-3470-4bad-b2ec-87341eeec530,2020-12-10,1,2020-12-10 23:59:57.308000+00:00,default,mobile,Chrome,Android,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
total_data.groupby('group').agg({'anonymous_id': 'count'})

,anonymous_id
group,
address_first,55707
default,445203


Для расчета метрик будет учитывать данные по пользователям, посетившим страницу лендинга

In [35]:
total_data.query('count_landing>0').groupby('group', as_index = False).agg({'anonymous_id': 'count'})\
                                    .rename(columns = {'anonymous_id': 'count'})

,group,count
0,address_first,23665
1,default,188045


In [36]:
188045 + 23665

211710

Добавим столбцы, характиризующие наличие отказа
bounce - True: не было посещение ни одной страницы, не считая лендинг
bounce_shop - True: не было посещение ни одной страницы, не считая лендинг и ввод адреса

и наличие факта добавления в корзину
product_add_ok - True: было добавление в корзину

In [37]:
total_data['bounce']=(total_data['count_mail']
                                 +total_data['count_address_change']
                                 +total_data['count_product_add']                      
                                 +total_data['count_shop_started']
                                 +total_data['count_shop_selected']
                                 +total_data['count_add_to_card']
                                 +total_data['count_order']
                                )==0

In [38]:
total_data['bounce_shop']=(total_data['count_mail']
                                 +total_data['count_product_add']
                                 +total_data['count_shop_started']
                                 +total_data['count_shop_selected']
                                 +total_data['count_order']
                                )==0

In [39]:
total_data['product_add_ok']=total_data['count_product_add']!=0

#### Расчет показателя bounce_rate (любая активность со страницы лендинга)

In [40]:
bounce_rate_table = pd.pivot_table(total_data.query('count_landing>0'), values='bounce', index=['group'],
                    columns=['device_type'], aggfunc=np.mean, margins=True).round(3)*100
bounce_rate_table

device_type,console,desktop,mobile,smarttv,tablet,All
group,,,,,,
address_first,100.0,54.1,51.8,0.0,65.4,53.1
default,62.5,54.0,54.2,57.1,60.7,54.2
All,66.7,54.0,53.9,50.0,61.3,54.1


In [41]:
bounce_rate_default = total_data.query('bounce==True and count_landing > 0 and group == "default"').bounce.count()/\
                total_data.query('group == "default" and count_landing > 0').bounce.count()
# проверяем что в таблице корректные данные
print ('bounce rate default = {:.1%}'.format(bounce_rate_default))

bounce rate default = 54.2%


In [42]:
bounce_rate_test = total_data.query('bounce==True and count_landing > 0 and group == "address_first"').bounce.count()/\
                total_data.query('group == "address_first" and count_landing > 0 ').bounce.count()
# проверяем что в таблице корректные данные
print ('bounce rate test = {:.1%}'.format(bounce_rate_test))

bounce rate test = 53.1%


Нулевая гипотеза Н0: bounce rate в тестовой и контрольной группе равны. Альтернативная гипотеза H1: bounce rate в тестовой группе и контрольной  группах отличаются

In [43]:
alpha = 0.05
clients_default = total_data.query('group == "default" and count_landing > 0').bounce
clients_test = total_data.query('group == "address_first" and count_landing > 0').bounce
result = st.ttest_ind(clients_default, clients_test)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=3.2015507744943483, pvalue=0.0013671016324971004)
Отвергаем нулевую гипотезу.


Нулевая гипотеза Н0: bounce rate при использовании задоново устройства в тестовой и контрольной группе равны. Альтернативная гипотеза H1: bounce rate в тестовой группе и контрольной  группах отличаются

In [44]:
device_type = "desktop"
alpha = 0.05
clients_default = total_data.query('group == "default" and count_landing > 0 and device_type == @device_type').bounce
clients_test = total_data.query('group == "address_first" and count_landing > 0 and  device_type == @device_type').bounce
result = st.ttest_ind(clients_default, clients_test)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=-0.23400706819848385, pvalue=0.8149799575180849)
Не получилось отвергнуть нулевую гипотезу


#### Расчет показателя конверсия в корзину 

In [45]:
rate_product_table = pd.pivot_table(total_data.query('count_landing>0'), values='product_add_ok', index=['group'],
                    columns=['device_type'], aggfunc=np.mean, margins=True).round(3)*100
rate_product_table

device_type,console,desktop,mobile,smarttv,tablet,All
group,,,,,,
address_first,0.0,21.0,9.2,0.0,8.9,14.8
default,0.0,19.9,6.1,28.6,9.5,12.8
All,0.0,20.0,6.4,25.0,9.5,13.1


In [46]:
rate_product_add_default = total_data.query('product_add_ok==True and count_landing > 0 and group == "default"')\
                                      .product_add_ok.count()/\
                            total_data.query('group == "default" and count_landing > 0').product_add_ok.count()
print ('Конверсия добавления товара в корзину в контрольной группе  = {:.1%}'.format(rate_product_add_default))
rate_product_add_test = total_data.query('product_add_ok==True and count_landing > 0 and group == "address_first"')\
                                      .product_add_ok.count()/\
                            total_data.query('group == "address_first" and count_landing > 0').product_add_ok.count()
print ('Конверсия добавления товара в корзину в тестовой  группе  = {:.1%}'.format(rate_product_add_test))

Конверсия добавления товара в корзину в контрольной группе  = 12.8%
Конверсия добавления товара в корзину в тестовой  группе  = 14.8%


In [47]:
alpha = 0.05
default = total_data.query('group == "default" and count_landing > 0').product_add_ok
test = total_data.query('group == "address_first" and count_landing > 0').product_add_ok
resultr = st.ttest_ind(default, test)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=-0.23400706819848385, pvalue=0.8149799575180849)
Не получилось отвергнуть нулевую гипотезу


#### Расчет показателя bounce_rate (без учета страницы ввода адреса)

In [48]:
bounce_rate_not_addr_table = pd.pivot_table(total_data.query('count_landing>0'), values='bounce_shop', index=['group'],
                    columns=['device_type'], aggfunc=np.mean, margins=True).round(3)*100
bounce_rate_not_addr_table

device_type,console,desktop,mobile,smarttv,tablet,All
group,,,,,,
address_first,100.0,60.1,68.2,0.0,72.1,64.4
default,62.5,54.1,54.4,57.1,60.8,54.4
All,66.7,54.8,56.0,50.0,62.1,55.5


In [49]:
bounce_rate_default_shop = total_data.query('bounce_shop==True and count_landing > 0 and group == "default"').bounce_shop.count()/\
                total_data.query('group == "default" and count_landing > 0').bounce_shop.count()
print ('bounce rate default only shop = {:.1%}'.format(bounce_rate_default_shop))

bounce rate default only shop = 54.4%


In [50]:
bounce_rate_test_shop = total_data.query('bounce_shop==True and count_landing > 0 and group == "address_first"').bounce_shop.count()/\
                total_data.query('group == "address_first" and count_landing > 0 ').bounce_shop.count()
print ('bounce rate test only shop = {:.1%}'.format(bounce_rate_test_shop))

bounce rate test only shop = 64.4%


Нулевая гипотеза Н0: bounce rate (без учета страницы ввода адреса) в тестовой и контрольной группе равны. Альтернативная гипотеза H1: bounce rate в тестовой группе и контрольной  группах отличаются

In [51]:
alpha = 0.05
clients_default = total_data.query('group == "default" and count_landing > 0').bounce_shop
clients_test = total_data.query('group == "address_first" and count_landing > 0').bounce_shop
result = st.ttest_ind(clients_default, clients_test)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=-29.183098451699657, pvalue=7.481706707097168e-187)
Отвергаем нулевую гипотезу.


Поиск аномалий

Есть пользователи, получавшие сообщение о том, что адрес «вне зоны доставки», хотя ни каких других активностей по пользователю не было. 

In [52]:
total_data.query('bounce == True and count_landing == 0 and count_not_zone > 0').value_counts('group')

group
default          242
address_first     31
dtype: int64

Есть пользователи, которые добавили товар в корзину, хотя попыток добавить товар в корзину не зафиксировано.

In [53]:
total_data.query('count_add_to_card == 0 and count_product_add > 0').value_counts('group')

group
default          140
address_first     23
dtype: int64

Есть пользователи одновременно зафиксированные и в тестовой и в контрольной группе.

In [62]:
client_group_count.query('group_count > 1')\
                              .merge(ab_test.drop_duplicates(subset=['date', 'anonymous_id', 'group'], keep='last'),\
                                     on = ['date', 'anonymous_id'], how = 'left')\
                              .sort_values('hit_at').head()

,anonymous_id,date,group_count,hit_at,group,device_type,browser,os
438,7a299bab-fefd-4de8-b82a-236e5f5f8dd4,2020-11-26,2,2020-11-26 02:42:10.302000+00:00,default,desktop,Chrome,Windows
439,7a299bab-fefd-4de8-b82a-236e5f5f8dd4,2020-11-26,2,2020-11-26 02:42:10.332000+00:00,address_first,desktop,Chrome,Windows
952,f18567dc-a005-42f1-b934-d2dab0cc1c41,2020-11-26,2,2020-11-26 02:47:50.250000+00:00,address_first,mobile,Chrome,Android
953,f18567dc-a005-42f1-b934-d2dab0cc1c41,2020-11-26,2,2020-11-26 02:47:50.432000+00:00,default,mobile,Chrome,Android
824,d3d51683-5df6-4803-ac64-a811e93dc6d5,2020-11-26,2,2020-11-26 02:57:02.111000+00:00,address_first,mobile,Chrome WebView,Android


In [63]:
1026/2

513.0

Частота посещений страниц лендинга

In [64]:
total_data[['anonymous_id','group', 'device_type','browser','os', 'count_landing', 'bounce']]\
        .sort_values('count_landing', ascending = False).head()

,anonymous_id,group,device_type,browser,os,count_landing,bounce
158045,6368eb82-d361-4aca-869e-080f167e3b5f,address_first,mobile,Chrome,Android,536.0,True
249268,8851d1da-4b18-4028-baa6-47e8a135fe87,default,desktop,Safari,Mac OS,150.0,False
352112,8da49d4a-48a7-4e27-8a16-5d122c9602c4,address_first,desktop,Yandex,Windows,79.0,False
130535,3e3c5ef8-fb2d-4c9b-8de2-224c78ce4149,default,desktop,Yandex,Windows,52.0,False
7287,4d74ffbd-a92d-4a49-bb2a-a5aa1ec3df79,default,desktop,Opera,Windows,50.0,False


Поиск закономерностей и изменений

Структура магазинов

In [65]:
shop = total_data.merge(retailer.groupby(['date', 'anonymous_id', 'retailer_id'], as_index = False).agg({'timestamp': 'count'})\
                , on = ['date', 'anonymous_id'], how = 'inner')[['date', 'anonymous_id', 'group', 'retailer_id']]
shop = pd.pivot_table(shop, values='anonymous_id', index=['retailer_id'],
                    columns=['group'], aggfunc='count')
shop['address_first_per']=(shop['address_first']/(shop['address_first'].sum()))*100
shop['default_per']=(shop['default']/(shop['default'].sum()))*100
shop.sort_values('default', ascending=False).head(10)

group,address_first,default,address_first_per,default_per
retailer_id,,,,
1,7009.0,59064.0,33.468628,28.499189
29,5275.0,49569.0,25.188616,23.917722
57,3743.0,34359.0,17.873174,16.578688
93,494.0,6309.0,2.358896,3.044179
33,516.0,5198.0,2.463948,2.508106
97,427.0,4285.0,2.038965,2.067571
193,27.0,4078.0,0.128928,1.967691
229,341.0,2891.0,1.628307,1.394947
61,276.0,2467.0,1.317926,1.190361


Связь количества добавлений товаров с суммой заказа

In [75]:
total_data = total_data.merge(order.groupby(['date', 'anonymous_id'], as_index = False).agg({'order_sum': 'sum'})\
                , on = ['date', 'anonymous_id'], how = 'left')

In [76]:
total_data[['count_product_add', 'order_sum']].corr()

,count_product_add,order_sum
count_product_add,1.000000,0.182478
order_sum,0.182478,1.000000


связь положительная слабая

In [77]:
total_data[['count_landing', 'bounce']].corr()

,count_landing,bounce
count_landing,1.00000,-0.14537
bounce,-0.14537,1.00000


связь отрицательная слабая

In [79]:
total_data.groupby(['date'], as_index = False).agg({'order_sum': 'sum'})

,date,order_sum
0,2020-11-26,4.064831e+06
1,2020-11-27,4.368580e+06
2,2020-11-28,3.773313e+06
3,2020-11-29,2.707209e+06
4,2020-11-30,2.561717e+06
5,2020-12-01,2.691321e+06
6,2020-12-02,3.230707e+06
7,2020-12-03,3.423439e+06
8,2020-12-04,4.454987e+06
9,2020-12-05,3.440876e+06


Проверим, что сумма покупки зависит от девайса или группы

In [80]:
pd.pivot_table(total_data, values='order_sum', index=['group'],
                    columns=['device_type'], aggfunc=np.mean, margins=True)

device_type,desktop,mobile,smarttv,tablet,All
group,,,,,
address_first,7501.908822,5552.347175,NaN,7577.295201,7144.405928
default,7285.404651,5820.335059,10020.099014,6873.425487,7048.823219
All,7309.814974,5785.097898,10020.099014,6938.732986,7059.858665


Проверим значимость отличий суммы покупки в типах девайса desktop и mobile

In [81]:
#H0: суммы равны
#H1: суммы статистически различаются
alpha = 0.05
desktop_order_sum = total_data.query('device_type == "desktop"').order_sum
mobile_order_sum = total_data.query('device_type == "mobile"').order_sum
result = st.ttest_ind(desktop_order_sum, mobile_order_sum)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=nan, pvalue=nan)
Не получилось отвергнуть нулевую гипотезу


In [82]:
#H0: суммы равны
#H1: суммы статистически различаются
alpha = 0.05
address_first = total_data.query('group == "address_first"').order_sum
default = total_data.query('group == "default"').order_sum
result = st.ttest_ind(address_first, default)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=nan, pvalue=nan)
Не получилось отвергнуть нулевую гипотезу


Количество заказов в засисимости от группы или девайса

In [83]:
pd.pivot_table(total_data.query('count_order>0'), values='count_order', index=['group'],
                    columns=['device_type'], aggfunc=np.mean, margins=True)

device_type,desktop,mobile,smarttv,tablet,All
group,,,,,
address_first,1.115616,1.473684,NaN,1.000000,1.180169
default,1.111811,1.349602,1.0,1.204545,1.150726
All,1.112240,1.365917,1.0,1.185567,1.154125


In [84]:
#H0: суммы равны
#H1: суммы статистически различаются
alpha = 0.05
address_first = total_data.query('group == "address_first"').query('count_order>0').count_order
default = total_data.query('group == "default"').query('count_order>0').count_order
result = st.ttest_ind(address_first, default)
print('p-значение:', result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

p-значение: Ttest_indResult(statistic=1.4563174729964774, pvalue=0.14534871887509554)
Не получилось отвергнуть нулевую гипотезу


Вспомогательные расчеты

Таблица для просмотра всех операций пользователя

In [85]:
ab_test2 = ab_test.rename(columns = {'hit_at': 'timestamp'})
ab_test2['table']= 'ab_test'
landing_viewed2 = landing_viewed
landing_viewed2['table']= 'landing_viewed'
retailer2 = retailer
retailer2['table']= 'retailer'
address_change2 = address_change
address_change2['table']= 'address_change'
message_not_zone2 = message_not_zone
message_not_zone2['table']= 'message_not_zone'
shop_started2 = shop_started
shop_started2['table']= 'shop_started'
shop_selected2 = shop_selected
shop_selected2['table']= 'shop_selected'
add_to_card2 = add_to_card
add_to_card2['table']= 'add_to_card'
product_add2 = product_add
product_add2['table']= 'product_add'
order2 = order
order2['table']= 'order'

In [86]:
res= pd.concat([ab_test2, landing_viewed2, retailer2, address_change2, message_not_zone2, shop_started2, shop_selected2,
                add_to_card2, product_add2, order2], ignore_index=True).sort_values('timestamp')

In [87]:
res_id = res.query('anonymous_id == "98201a66-3667-48f1-96e7-42ce28d9b959"')
res_id.head()

,timestamp,anonymous_id,group,device_type,browser,os,date,table,retailer_id,source,order_sum
1476681,2020-11-26 05:38:27.551000+00:00,98201a66-3667-48f1-96e7-42ce28d9b959,NaN,NaN,NaN,NaN,2020-11-26,address_change,NaN,address,NaN
1710564,2020-11-26 05:38:42.526000+00:00,98201a66-3667-48f1-96e7-42ce28d9b959,NaN,NaN,NaN,NaN,2020-11-26,shop_selected,NaN,header,NaN
974184,2020-11-26 05:38:45.350000+00:00,98201a66-3667-48f1-96e7-42ce28d9b959,NaN,NaN,NaN,NaN,2020-11-26,retailer,1.0,NaN,NaN
1476677,2020-11-26 05:40:21.120000+00:00,98201a66-3667-48f1-96e7-42ce28d9b959,NaN,NaN,NaN,NaN,2020-11-26,address_change,NaN,address,NaN
1710573,2020-11-26 08:01:24.581000+00:00,98201a66-3667-48f1-96e7-42ce28d9b959,NaN,NaN,NaN,NaN,2020-11-26,shop_selected,NaN,header,NaN


In [88]:
# Specify a writer
writer = pd.ExcelWriter('example.xlsx', engine='xlsxwriter')

# Write your DataFrame to a file     
res_id.astype({"timestamp": str}).to_excel(writer, 'Sheet1')

# Save the result 
writer.save()